In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
training_data = datasets.FashionMNIST(root="data",train=True,download=True,transform=ToTensor())
testing_data = datasets.FashionMNIST(root="data",train=False,download=True,transform=ToTensor())

In [6]:
batch_size = 64

training_data_loader = DataLoader(training_data,batch_size=batch_size)
testing_data_loader = DataLoader(testing_data,batch_size=batch_size)

for X,y in testing_data_loader:
    print(f"Shape of X[N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X[N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [7]:
device = ('cuda' if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"using device {device}")

using device cpu


In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [24]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [25]:
def train(dataloader, model:nn.Module, loss_fn:nn.CrossEntropyLoss, optimizer):
    size = len(dataloader.dataset)
    model.train() # sets module in training mode
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)

        #predict and get the error
        pred = model(X)
        loss = loss_fn(pred,y)

        #backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [26]:
def test(dataloader, model:nn.Module, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # sets module in evaluation mode
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [27]:
epochs = 5
for t in range(epochs):
    
    print(f"Epoch {t+1}\n-------------------------------")
    train(training_data_loader, model, loss_fn, optimizer)
    test(testing_data_loader, model, loss_fn)

print("Done!")

Epoch 1
-------------------------------
loss: 2.299724  [   64/60000]
loss: 2.291102  [ 6464/60000]
loss: 2.274094  [12864/60000]
loss: 2.266974  [19264/60000]
loss: 2.249399  [25664/60000]
loss: 2.226276  [32064/60000]
loss: 2.227283  [38464/60000]
loss: 2.192272  [44864/60000]
loss: 2.183540  [51264/60000]
loss: 2.154141  [57664/60000]
Test Error: 
 Accuracy: 47.2%, Avg loss: 2.157632 

Epoch 2
-------------------------------
loss: 2.164852  [   64/60000]
loss: 2.162076  [ 6464/60000]
loss: 2.105527  [12864/60000]
loss: 2.117648  [19264/60000]
loss: 2.067086  [25664/60000]
loss: 2.004032  [32064/60000]
loss: 2.031563  [38464/60000]
loss: 1.949140  [44864/60000]
loss: 1.950905  [51264/60000]
loss: 1.876613  [57664/60000]
Test Error: 
 Accuracy: 53.1%, Avg loss: 1.888081 

Epoch 3
-------------------------------
loss: 1.919366  [   64/60000]
loss: 1.898294  [ 6464/60000]
loss: 1.781851  [12864/60000]
loss: 1.815315  [19264/60000]
loss: 1.703098  [25664/60000]
loss: 1.652561  [32064/600

In [29]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [10]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x,y = testing_data[0][0], testing_data[0][1]
with torch.no_grad():
    pred = model(x)
    print(pred)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


tensor([[-2.4318, -2.6873, -1.1547, -2.0186, -1.0958,  2.2845, -1.1788,  2.5103,
          1.6418,  3.0134]])
Predicted: "Ankle boot", Actual: "Ankle boot"
